In [ ]:
#!pip install etherscan-python

In [ ]:
#!pip3 install mysql-connector

In [1]:
import importlib_resources as resources
import sys
import pandas as pd
import threading
from datetime import datetime, timedelta
import os
import numpy as np

from etherscan import Etherscan
import json
import time
import requests

import mysql.connector

#### Connect to DB

In [2]:
conn = mysql.connector.MySQLConnection(user='admin', password='wWusLXWEsxNqaviwGPsP',
                                 host='cryptologic-test-mysql-db.cyage1xxew24.us-east-1.rds.amazonaws.com',
                                 database='cryptologic_BE_Dev')


#### Wallet Table

- Get Wallets from BD

In [3]:
wllt = pd.read_sql('SELECT * FROM WALLET', con=conn)
wllt.head()

,index,Wallet_Hash,Balance,Update_Date,Coin_Type,Load_Date
0,6,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,301.841575,2022-07-24 19:25:16,eth,2022-07-16 11:25:00
1,7,0x4be353d6601b51e4ee7a8850c9a0f8f289d8da42,0.000026,2022-07-24 19:25:16,eth,2022-07-16 11:25:00
2,5040,0xf1090f0adceed80b84cbca7649281c71cff9c728,NaN,NaT,None,2022-07-30 21:19:40
3,5041,0xc0b4189e4707da39e5e3ee2f388528623522e36f,NaN,NaT,None,2022-07-30 21:19:40
4,5042,0x599872bf0ba7689a4a71805b5f4d09fb275df5c0,NaN,NaT,None,2022-07-30 21:19:40


In [4]:
wllt = wllt[wllt['Balance'].notna()]
wllt.head()

,index,Wallet_Hash,Balance,Update_Date,Coin_Type,Load_Date
0,6,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,301.841575,2022-07-24 19:25:16,eth,2022-07-16 11:25:00
1,7,0x4be353d6601b51e4ee7a8850c9a0f8f289d8da42,0.000026,2022-07-24 19:25:16,eth,2022-07-16 11:25:00
11283,16321,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,249.248420,2022-08-01 20:16:10,eth,2022-07-16 11:25:00
11284,16322,0x4be353d6601b51e4ee7a8850c9a0f8f289d8da42,0.000026,2022-08-01 20:16:10,eth,2022-07-16 11:25:00
11285,16323,0xf1090f0adceed80b84cbca7649281c71cff9c728,0.000000,2022-08-01 20:16:10,eth,2022-07-30 21:19:40


#### Trx Table

In [5]:
trx = pd.read_sql('SELECT * FROM TRANSACTIONS', con=conn)
trx.head()

,Trx_From_Wallet,Trx_To_Wallet,Trx_Amount,Trx_Datetime,Trx_Hash,Trx_Gas,Trx_GasPrice,Trx_Status,Trx_Method_ID
0,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,0x8ec45f94832f5ff2e92bf2237b921526b5b15d0f,0.127408,2015-08-19 09:19:07,0xf2170b178010ca3f3f9cdab6e3873ec614fe996485e5...,2.500000e-15,1.146890e-07,,0x
1,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,0xd9c13364c95e5f8e35d4bd4830e974a0652f83d8,0.060884,2015-08-19 09:19:07,0xa85d9c35ec92e144f191e68d888a0c990c7046afcf47...,2.500000e-15,1.146890e-07,,0x
2,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,0xfe75342791a8492e5bfef9dbe16d4ec937671c9d,0.028327,2015-08-19 09:19:07,0x7b0938ad58590e4d080a388c5e32b19bfca994c3dbb4...,2.500000e-15,1.146890e-07,,0x
3,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,0x37aad32f4b316d18a23db46bd3531d7d4d5fca8a,0.426569,2015-08-19 09:19:07,0x86208990131e66b63cfdeae5742a608756679052a216...,2.500000e-15,1.146890e-07,,0x
4,0x52bc44d5378309ee2abf1539bf71de1b7d7be3b5,0xf4f2c15602b084cae84ea603f75527de19705aa1,0.106135,2015-08-19 09:19:07,0x2c4dc63c9cd164158562d19076c9e6d73d69f01c71c6...,2.500000e-15,1.146890e-07,,0x


In [15]:
def computed_delta_trx(trx, agg_hash, delta_feat, suffix, agg_type_lst_min=[np.mean, np.min], agg_type_lst_day=[np.mean, np.max], minutes=True, day=True):

    #Order Datafram by wallet and date of trx
    trx.sort_values(by=[agg_hash, delta_feat], inplace=True, ascending = [True, True])

    min_var_name = 'min_'+suffix+'_trx_'
    day_var_name = 'day_'+suffix+'_trx_'

    #Get delta between row n-1 and row n
    mask = trx.duplicated([agg_hash])
    trx[min_var_name] = (np.where(mask, (trx[delta_feat] - 
                                         trx[delta_feat].shift(1)).dt.total_seconds()/60, np.nan))
    trx[day_var_name] = (np.where(mask, (trx[delta_feat] - 
                                         trx[delta_feat].shift(1)).dt.days, np.nan))

    #Group by Wallet hash to calc Avg and Min
    features = trx.groupby([agg_hash]).agg({min_var_name:agg_type_lst_min, day_var_name:agg_type_lst_day})

    #Drop Column Multi levels
    features.columns = [''.join(col) for col in features.columns]
    
    features = features.reset_index()

    return features

- Get **Avg minutes** between **sent transactions**

In [16]:
features = computed_delta_trx(trx, 'Trx_From_Wallet', 'Trx_Datetime', 'sent')

features.head()

,Trx_From_Wallet,min_sent_trx_mean,min_sent_trx_amin,day_sent_trx_mean,day_sent_trx_amax
0,0x000634de1a6144f8427a8c1365b363becc8799db,NaN,NaN,NaN,NaN
1,0x000798843c29cc80b7ddebbfab1af39458cdd02f,NaN,NaN,NaN,NaN
2,0x00192fb10df37c9fb26829eb2cc623cd1bf599e8,1648.259825,93.833333,0.842105,53.0
3,0x001ecc43a3c2c25742ebc7442479ee608aa748de,NaN,NaN,NaN,NaN
4,0x002747c0717f03d457634337b0caafb1994dad91,NaN,NaN,NaN,NaN


- Get **Avg minutes** between **received transactions**

In [17]:
ftr = computed_delta_trx(trx, 'Trx_To_Wallet', 'Trx_Datetime', 'rec')
ftr.head()

,Trx_To_Wallet,min_rec_trx_mean,min_rec_trx_amin,day_rec_trx_mean,day_rec_trx_amax
0,,20.333333,20.333333,0.000000,0.0
1,0x0000000000000000000000000000000000000000,57794.544444,0.000000,39.666667,109.0
2,0x00049aab94a30672a8e382220db9ff25422453b4,99.733333,99.733333,0.000000,0.0
3,0x00055c8c6ead6deb14edf133b98656de4745aca4,NaN,NaN,NaN,NaN
4,0x0006b0b6df850a30e75970131478afee218f30e0,NaN,NaN,NaN,NaN


In [41]:
#Sent trx
ftr_min = trx.groupby(['Trx_From_Wallet']).agg({'Trx_Datetime':'min', 'Trx_Amount':'min'})\
                    .rename(columns={'Trx_Datetime':'Min_sent_Trx_Datetime', 'Trx_Amount':'Min_sent_Trx_Amount'})
ftr_max = trx.groupby(['Trx_From_Wallet']).agg({'Trx_Datetime':'max', 'Trx_Amount':'max'})\
                    .rename(columns={'Trx_Datetime':'Max_sent_Trx_Datetime', 'Trx_Amount':'Max_sent_Trx_Amount'})

df_sent = pd.merge(ftr_min, ftr_max, on='Trx_From_Wallet', how='outer').reset_index()

#Received Trx
ftr_min = trx.groupby(['Trx_To_Wallet']).agg({'Trx_Datetime':'min', 'Trx_Amount':'min'})\
                    .rename(columns={'Trx_Datetime':'Min_rec_Trx_Datetime', 'Trx_Amount':'Min_rec_Trx_Amount'})
ftr_max = trx.groupby(['Trx_To_Wallet']).agg({'Trx_Datetime':'max', 'Trx_Amount':'max'})\
                    .rename(columns={'Trx_Datetime':'Max_rec_Trx_Datetime', 'Trx_Amount':'Max_rec_Trx_Amount'})

df_rcvd = pd.merge(ftr_min, ftr_max, on='Trx_To_Wallet', how='outer').reset_index()

#Merge both df
df_aux = pd.merge(df_sent, df_rcvd, left_on='Trx_From_Wallet', right_on='Trx_To_Wallet', how='outer')

df_aux['Time_diff_sent_frst_lst_trx'] = df_aux['Max_sent_Trx_Datetime'] - df_aux['Min_sent_Trx_Datetime']

df_aux['Time_diff_rec_frst_lst_trx'] = df_aux['Max_rec_Trx_Datetime'] - df_aux['Min_rec_Trx_Datetime']

df_aux[df_aux['Trx_From_Wallet'].notnull() & df_aux['Trx_To_Wallet'].notnull()]

,Trx_From_Wallet,Min_sent_Trx_Datetime,Min_sent_Trx_Amount,Max_sent_Trx_Datetime,Max_sent_Trx_Amount,Trx_To_Wallet,Min_rec_Trx_Datetime,Min_rec_Trx_Amount,Max_rec_Trx_Datetime,Max_rec_Trx_Amount,Time_diff_sent_frst_lst_trx,Time_diff_rec_frst_lst_trx
13,0x007aee65ac559b198140b541b053ec79cbdb50c5,2015-09-17 20:04:24,0.330000,2016-05-15 21:53:42,1.500000,0x007aee65ac559b198140b541b053ec79cbdb50c5,2015-08-22 15:32:11,0.002960,2015-10-25 08:55:24,0.079316,241 days 01:49:18,63 days 17:23:13
18,0x008bd9521a1b9de4c9d969b01997d737042c9921,2016-03-28 02:09:09,0.066160,2016-03-28 02:09:09,0.066160,0x008bd9521a1b9de4c9d969b01997d737042c9921,2015-08-27 11:48:25,0.010044,2016-02-28 05:06:20,0.035902,0 days 00:00:00,184 days 17:17:55
21,0x0096056d282b0ca9571987dc1264aeb8ec0b80c6,2016-03-28 02:14:01,0.051778,2016-03-28 03:07:31,0.100000,0x0096056d282b0ca9571987dc1264aeb8ec0b80c6,2015-08-27 11:46:53,0.010590,2016-03-28 02:57:49,0.100084,0 days 00:53:30,213 days 15:10:56
28,0x00cd41b1190dc58043d562869849f1a6410025b7,2016-05-11 16:14:23,0.043606,2017-12-03 22:21:05,134.501495,0x00cd41b1190dc58043d562869849f1a6410025b7,2015-08-30 10:48:30,0.029654,2017-12-03 22:19:01,0.590860,571 days 06:06:42,826 days 11:30:31
78,0x022a006fcc619ebf387f413ef06932babd47ae0e,2015-08-12 12:46:52,0.010117,2021-02-05 03:53:31,2.866094,0x022a006fcc619ebf387f413ef06932babd47ae0e,2015-08-12 12:39:37,0.010374,2021-02-05 03:49:29,2.866220,2003 days 15:06:39,2003 days 15:09:52
...,...,...,...,...,...,...,...,...,...,...,...,...
9981,0xfd60ac5e5b5eced795b1eec5403b3ab803a25980,2021-04-30 01:09:01,0.261422,2021-04-30 01:09:01,0.261422,0xfd60ac5e5b5eced795b1eec5403b3ab803a25980,2021-04-30 01:07:40,0.261537,2021-04-30 01:07:40,0.261537,0 days 00:00:00,0 days 00:00:00
10039,0xfeb5e65bf964aee62bfbc7c4840f24a5f09591d7,2016-05-11 16:14:23,0.019965,2021-01-13 20:20:54,92.297326,0xfeb5e65bf964aee62bfbc7c4840f24a5f09591d7,2015-08-31 09:32:18,0.011005,2021-01-13 20:20:21,12.561034,1708 days 04:06:31,1962 days 10:48:03
10064,0xff2223f9da872e3a92d286330e26f16ea371f6bf,2015-08-31 09:51:01,0.010931,2018-10-11 20:34:07,0.510343,0xff2223f9da872e3a92d286330e26f16ea371f6bf,2015-08-31 09:32:11,0.001512,2018-10-11 20:27:21,0.510469,1137 days 10:43:06,1137 days 10:55:10
10083,0xff78baa804add29cd783483b6ccd150b0589ddcb,2015-08-27 07:14:00,0.015382,2019-07-16 04:58:04,20.015193,0xff78baa804add29cd783483b6ccd150b0589ddcb,2015-08-27 07:08:33,0.001998,2019-07-16 04:55:52,20.011000,1418 days 21:44:04,1418 days 21:47:19


- Close Connection

In [ ]:
conn.close()